In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import pickle
from os.path import join, exists
import argparse
from tqdm import tqdm
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms, utils
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# Any results you write to the current directory are saved as output.

In [ ]:
print(os.listdir('../input'))

First, let's check the number of classes there are in our dataset

In [ ]:
classes = os.listdir('../input/lsa64-hand-gesture-recognition-dataset/lsa40/LSA40/train')
print(len(classes))

We need to have the classes and video names in a DataFrame so that our dataloader can access it easily. 

In [ ]:
dataset = []
for dirname, _, filenames in os.walk('/kaggle/input/lsa64-hand-gesture-recognition-dataset/lsa40/LSA40/train'):

    for filename in filenames:
        dataset.append([dirname[71:],filename])

print(len(dataset))

df = pd.DataFrame(dataset,columns=['Gesture','File'])
df.head()

We need to encode the categorical values in Gesture column to numerical values so that our model can use it

In [ ]:
#TODO: Setting categorical values to integers starting from 0
df['Folder'] = df['Gesture']
encode_dict = {k: v for v, k in enumerate(classes)}
df.replace({"Gesture": encode_dict},inplace=True)
df.head()

# Splitting in Training and Validation Data

In [ ]:
train_df,valid_df = np.split(df, [int(.8*len(df))])

# Configuration Class

In [ ]:
class Config():
    rootDir = '../input/lsa64-hand-gesture-recognition-dataset/lsa40/LSA40/train/'
    batch_size = 8
    train_number_epochs = 10
    num_workers = 8

# Dataset

On exploring the dataset, it is found that for the videos in training dataset, minimum number of frames is 58 and maximum number of frames is 212. So, we'll work on an input size of 200 frames

In [ ]:
class videoDataset(Dataset):
    """Dataset Class for Loading Video"""

    def __init__(self, df, rootDir, channels, timeDepth, xSize, ySize, transform=None):
        """
        Args:
            clipsList (string): Path to the clipsList file with labels.
            rootDir (string): Directory with all the videoes.
            transform (callable, optional): Optional transform to be applied
                    on a sample.
            channels: Number of channels of frames
            timeDepth: Number of frames to be loaded in a sample
            xSize, ySize: Dimensions of the frames
            mean: Mean valuse of the training set videos over each channel
        """
        self.df = df
        self.rootDir = rootDir
        self.channels = channels
        self.timeDepth = timeDepth
        self.xSize = xSize
        self.ySize = ySize
        self.transform = transform
        self.length = 0

    def __len__(self):
        return len(self.df)
    
    def handsegment(self,frame):
        
        boundaries = [([0, 120, 0], [140, 255, 100]),
                        ([25, 0, 75], [180, 38, 255])]
        
        lower, upper = boundaries[0]
        lower = np.array(lower, dtype="uint8")
        upper = np.array(upper, dtype="uint8")
        mask1 = cv2.inRange(frame, lower, upper)

        lower, upper = boundaries[1]
        lower = np.array(lower, dtype="uint8")
        upper = np.array(upper, dtype="uint8")
        mask2 = cv2.inRange(frame, lower, upper)

        mask = cv2.bitwise_or(mask1, mask2)
        output = cv2.bitwise_and(frame, frame, mask=mask)
        return output

    def readVideo(self, videoFilePath):
        # Open the video file
        cap = cv2.VideoCapture(videoFilePath)
        self.length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        # nFrames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frames = torch.zeros(self.timeDepth,self.channels,self.xSize,self.ySize)
        
        f=0
        while f<self.timeDepth:
            ret, frame = cap.read()
            if ret:
                frame = self.handsegment(frame)
                frame = torch.from_numpy(frame)
                frame = frame.permute(2, 1, 0)
                # HWC2CHW
                if self.transform:
                    frame = self.transform(frame)
                frames[f,:, :, :] = frame
            else:
                
                break
            f+=1

#         for c in range(3):
#             frames[c] -= self.mean[c]
#         frames /= 255
        return frames

    def __getitem__(self, idx):
        label = self.df.iloc[idx,0]
        fileName = self.df.iloc[idx,1]
        folder = self.df.iloc[idx,2]
        
        videoFilePath = self.rootDir + folder +'/' + fileName
        clip = self.readVideo(videoFilePath)
        
        sample = {'clip': clip, 'label': label}

        return clip,label
#         cap = cv2.VideoCapture(videoFilePath)
#         length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
#         return length

In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

train_dataset = videoDataset(df=train_df, rootDir=Config.rootDir, channels=3, timeDepth=200, xSize=299, ySize=299, transform = transform)
valid_dataset = videoDataset(df=valid_df, rootDir=Config.rootDir, channels=3, timeDepth=200, xSize=299, ySize=299, transform = transform)
#train_loader = torch.utils.data.DataLoader(dataset,shuffle=True)

# Dataloaders

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset,shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset,shuffle=True)

# Helper Function

In [ ]:
def imshow(img,text=None,should_save=False):
    npimg = img.numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()    

def show_plot(iteration,loss):
    plt.plot(iteration,loss)
    plt.show()

In [ ]:
clip = train_dataset.__getitem__(0)[0]
imshow(clip[0])

# Check CUDA Availability

In [ ]:
#Checking if CUDA is available or not
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

# CNN Encoder - Inception v3

Our idea is to take the video, split into frames, and then feed these frames to RNN. These individual frames are input into Inception pretrained model to get spatial information.
**Input size for each image** = 3 x 299 x 299
**Output** = 1 x 1000


In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
class Inception_v3_pt_mcn(nn.Module):

    def __init__(self):
        super(Inception_v3_pt_mcn, self).__init__()
        self.meta = {'mean': [0.485, 0.456, 0.406],
                     'std': [0.229, 0.224, 0.225],
                     'imageSize': [299, 299]}
        self.features_0_conv = nn.Conv2d(3, 32, kernel_size=[3, 3], stride=(2, 2), bias=False)
        self.features_0_bn = nn.BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True)
        self.features_0_relu = nn.ReLU()
        self.features_1_conv = nn.Conv2d(32, 32, kernel_size=[3, 3], stride=(1, 1), bias=False)
        self.features_1_bn = nn.BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True)
        self.features_1_relu = nn.ReLU()
        self.features_2_conv = nn.Conv2d(32, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1), bias=False)
        self.features_2_bn = nn.BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True)
        self.features_2_relu = nn.ReLU()
        self.features_3 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
        self.features_4_conv = nn.Conv2d(64, 80, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_4_bn = nn.BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True)
        self.features_4_relu = nn.ReLU()
        self.features_5_conv = nn.Conv2d(80, 192, kernel_size=[3, 3], stride=(1, 1), bias=False)
        self.features_5_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_5_relu = nn.ReLU()
        self.features_6 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
        self.features_7_branch1x1_conv = nn.Conv2d(192, 64, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_7_branch1x1_bn = nn.BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True)
        self.features_7_branch1x1_relu = nn.ReLU()
        self.features_7_branch5x5_1_conv = nn.Conv2d(192, 48, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_7_branch5x5_1_bn = nn.BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True)
        self.features_7_branch5x5_1_relu = nn.ReLU()
        self.features_7_branch5x5_2_conv = nn.Conv2d(48, 64, kernel_size=[5, 5], stride=(1, 1), padding=(2, 2), bias=False)
        self.features_7_branch5x5_2_bn = nn.BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True)
        self.features_7_branch5x5_2_relu = nn.ReLU()
        self.features_7_branch3x3dbl_1_conv = nn.Conv2d(192, 64, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_7_branch3x3dbl_1_bn = nn.BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True)
        self.features_7_branch3x3dbl_1_relu = nn.ReLU()
        self.features_7_branch3x3dbl_2_conv = nn.Conv2d(64, 96, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1), bias=False)
        self.features_7_branch3x3dbl_2_bn = nn.BatchNorm2d(96, eps=0.001, momentum=0.1, affine=True)
        self.features_7_branch3x3dbl_2_relu = nn.ReLU()
        self.features_7_branch3x3dbl_3_conv = nn.Conv2d(96, 96, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1), bias=False)
        self.features_7_branch3x3dbl_3_bn = nn.BatchNorm2d(96, eps=0.001, momentum=0.1, affine=True)
        self.features_7_branch3x3dbl_3_relu = nn.ReLU()
        self.features_7_branch_avgpool = nn.AvgPool2d(kernel_size=[3, 3], stride=[1, 1], padding=1, ceil_mode=False, count_include_pad=False)
        self.features_7_branch_pool_conv = nn.Conv2d(192, 32, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_7_branch_pool_bn = nn.BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True)
        self.features_7_branch_pool_relu = nn.ReLU()
        self.features_8_branch1x1_conv = nn.Conv2d(256, 64, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_8_branch1x1_bn = nn.BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True)
        self.features_8_branch1x1_relu = nn.ReLU()
        self.features_8_branch5x5_1_conv = nn.Conv2d(256, 48, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_8_branch5x5_1_bn = nn.BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True)
        self.features_8_branch5x5_1_relu = nn.ReLU()
        self.features_8_branch5x5_2_conv = nn.Conv2d(48, 64, kernel_size=[5, 5], stride=(1, 1), padding=(2, 2), bias=False)
        self.features_8_branch5x5_2_bn = nn.BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True)
        self.features_8_branch5x5_2_relu = nn.ReLU()
        self.features_8_branch3x3dbl_1_conv = nn.Conv2d(256, 64, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_8_branch3x3dbl_1_bn = nn.BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True)
        self.features_8_branch3x3dbl_1_relu = nn.ReLU()
        self.features_8_branch3x3dbl_2_conv = nn.Conv2d(64, 96, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1), bias=False)
        self.features_8_branch3x3dbl_2_bn = nn.BatchNorm2d(96, eps=0.001, momentum=0.1, affine=True)
        self.features_8_branch3x3dbl_2_relu = nn.ReLU()
        self.features_8_branch3x3dbl_3_conv = nn.Conv2d(96, 96, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1), bias=False)
        self.features_8_branch3x3dbl_3_bn = nn.BatchNorm2d(96, eps=0.001, momentum=0.1, affine=True)
        self.features_8_branch3x3dbl_3_relu = nn.ReLU()
        self.features_8_branch_avgpool = nn.AvgPool2d(kernel_size=[3, 3], stride=[1, 1], padding=1, ceil_mode=False, count_include_pad=False)
        self.features_8_branch_pool_conv = nn.Conv2d(256, 64, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_8_branch_pool_bn = nn.BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True)
        self.features_8_branch_pool_relu = nn.ReLU()
        self.features_9_branch1x1_conv = nn.Conv2d(288, 64, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_9_branch1x1_bn = nn.BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True)
        self.features_9_branch1x1_relu = nn.ReLU()
        self.features_9_branch5x5_1_conv = nn.Conv2d(288, 48, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_9_branch5x5_1_bn = nn.BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True)
        self.features_9_branch5x5_1_relu = nn.ReLU()
        self.features_9_branch5x5_2_conv = nn.Conv2d(48, 64, kernel_size=[5, 5], stride=(1, 1), padding=(2, 2), bias=False)
        self.features_9_branch5x5_2_bn = nn.BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True)
        self.features_9_branch5x5_2_relu = nn.ReLU()
        self.features_9_branch3x3dbl_1_conv = nn.Conv2d(288, 64, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_9_branch3x3dbl_1_bn = nn.BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True)
        self.features_9_branch3x3dbl_1_relu = nn.ReLU()
        self.features_9_branch3x3dbl_2_conv = nn.Conv2d(64, 96, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1), bias=False)
        self.features_9_branch3x3dbl_2_bn = nn.BatchNorm2d(96, eps=0.001, momentum=0.1, affine=True)
        self.features_9_branch3x3dbl_2_relu = nn.ReLU()
        self.features_9_branch3x3dbl_3_conv = nn.Conv2d(96, 96, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1), bias=False)
        self.features_9_branch3x3dbl_3_bn = nn.BatchNorm2d(96, eps=0.001, momentum=0.1, affine=True)
        self.features_9_branch3x3dbl_3_relu = nn.ReLU()
        self.features_9_branch_avgpool = nn.AvgPool2d(kernel_size=[3, 3], stride=[1, 1], padding=1, ceil_mode=False, count_include_pad=False)
        self.features_9_branch_pool_conv = nn.Conv2d(288, 64, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_9_branch_pool_bn = nn.BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True)
        self.features_9_branch_pool_relu = nn.ReLU()
        self.features_10_branch3x3_conv = nn.Conv2d(288, 384, kernel_size=[3, 3], stride=(2, 2), bias=False)
        self.features_10_branch3x3_bn = nn.BatchNorm2d(384, eps=0.001, momentum=0.1, affine=True)
        self.features_10_branch3x3_relu = nn.ReLU()
        self.features_10_branch3x3dbl_1_conv = nn.Conv2d(288, 64, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_10_branch3x3dbl_1_bn = nn.BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True)
        self.features_10_branch3x3dbl_1_relu = nn.ReLU()
        self.features_10_branch3x3dbl_2_conv = nn.Conv2d(64, 96, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1), bias=False)
        self.features_10_branch3x3dbl_2_bn = nn.BatchNorm2d(96, eps=0.001, momentum=0.1, affine=True)
        self.features_10_branch3x3dbl_2_relu = nn.ReLU()
        self.features_10_branch3x3dbl_3_conv = nn.Conv2d(96, 96, kernel_size=[3, 3], stride=(2, 2), bias=False)
        self.features_10_branch3x3dbl_3_bn = nn.BatchNorm2d(96, eps=0.001, momentum=0.1, affine=True)
        self.features_10_branch3x3dbl_3_relu = nn.ReLU()
        self.features_10_branch_pool = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
        self.features_11_branch1x1_conv = nn.Conv2d(768, 192, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_11_branch1x1_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_11_branch1x1_relu = nn.ReLU()
        self.features_11_branch7x7_1_conv = nn.Conv2d(768, 128, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_11_branch7x7_1_bn = nn.BatchNorm2d(128, eps=0.001, momentum=0.1, affine=True)
        self.features_11_branch7x7_1_relu = nn.ReLU()
        self.features_11_branch7x7_2_conv = nn.Conv2d(128, 128, kernel_size=[1, 7], stride=(1, 1), padding=(0, 3), bias=False)
        self.features_11_branch7x7_2_bn = nn.BatchNorm2d(128, eps=0.001, momentum=0.1, affine=True)
        self.features_11_branch7x7_2_relu = nn.ReLU()
        self.features_11_branch7x7_3_conv = nn.Conv2d(128, 192, kernel_size=[7, 1], stride=(1, 1), padding=(3, 0), bias=False)
        self.features_11_branch7x7_3_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_11_branch7x7_3_relu = nn.ReLU()
        self.features_11_branch7x7dbl_1_conv = nn.Conv2d(768, 128, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_11_branch7x7dbl_1_bn = nn.BatchNorm2d(128, eps=0.001, momentum=0.1, affine=True)
        self.features_11_branch7x7dbl_1_relu = nn.ReLU()
        self.features_11_branch7x7dbl_2_conv = nn.Conv2d(128, 128, kernel_size=[7, 1], stride=(1, 1), padding=(3, 0), bias=False)
        self.features_11_branch7x7dbl_2_bn = nn.BatchNorm2d(128, eps=0.001, momentum=0.1, affine=True)
        self.features_11_branch7x7dbl_2_relu = nn.ReLU()
        self.features_11_branch7x7dbl_3_conv = nn.Conv2d(128, 128, kernel_size=[1, 7], stride=(1, 1), padding=(0, 3), bias=False)
        self.features_11_branch7x7dbl_3_bn = nn.BatchNorm2d(128, eps=0.001, momentum=0.1, affine=True)
        self.features_11_branch7x7dbl_3_relu = nn.ReLU()
        self.features_11_branch7x7dbl_4_conv = nn.Conv2d(128, 128, kernel_size=[7, 1], stride=(1, 1), padding=(3, 0), bias=False)
        self.features_11_branch7x7dbl_4_bn = nn.BatchNorm2d(128, eps=0.001, momentum=0.1, affine=True)
        self.features_11_branch7x7dbl_4_relu = nn.ReLU()
        self.features_11_branch7x7dbl_5_conv = nn.Conv2d(128, 192, kernel_size=[1, 7], stride=(1, 1), padding=(0, 3), bias=False)
        self.features_11_branch7x7dbl_5_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_11_branch7x7dbl_5_relu = nn.ReLU()
        self.features_11_branch_avgpool = nn.AvgPool2d(kernel_size=[3, 3], stride=[1, 1], padding=1, ceil_mode=False, count_include_pad=False)
        self.features_11_branch_pool_conv = nn.Conv2d(768, 192, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_11_branch_pool_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_11_branch_pool_relu = nn.ReLU()
        self.features_12_branch1x1_conv = nn.Conv2d(768, 192, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_12_branch1x1_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_12_branch1x1_relu = nn.ReLU()
        self.features_12_branch7x7_1_conv = nn.Conv2d(768, 160, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_12_branch7x7_1_bn = nn.BatchNorm2d(160, eps=0.001, momentum=0.1, affine=True)
        self.features_12_branch7x7_1_relu = nn.ReLU()
        self.features_12_branch7x7_2_conv = nn.Conv2d(160, 160, kernel_size=[1, 7], stride=(1, 1), padding=(0, 3), bias=False)
        self.features_12_branch7x7_2_bn = nn.BatchNorm2d(160, eps=0.001, momentum=0.1, affine=True)
        self.features_12_branch7x7_2_relu = nn.ReLU()
        self.features_12_branch7x7_3_conv = nn.Conv2d(160, 192, kernel_size=[7, 1], stride=(1, 1), padding=(3, 0), bias=False)
        self.features_12_branch7x7_3_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_12_branch7x7_3_relu = nn.ReLU()
        self.features_12_branch7x7dbl_1_conv = nn.Conv2d(768, 160, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_12_branch7x7dbl_1_bn = nn.BatchNorm2d(160, eps=0.001, momentum=0.1, affine=True)
        self.features_12_branch7x7dbl_1_relu = nn.ReLU()
        self.features_12_branch7x7dbl_2_conv = nn.Conv2d(160, 160, kernel_size=[7, 1], stride=(1, 1), padding=(3, 0), bias=False)
        self.features_12_branch7x7dbl_2_bn = nn.BatchNorm2d(160, eps=0.001, momentum=0.1, affine=True)
        self.features_12_branch7x7dbl_2_relu = nn.ReLU()
        self.features_12_branch7x7dbl_3_conv = nn.Conv2d(160, 160, kernel_size=[1, 7], stride=(1, 1), padding=(0, 3), bias=False)
        self.features_12_branch7x7dbl_3_bn = nn.BatchNorm2d(160, eps=0.001, momentum=0.1, affine=True)
        self.features_12_branch7x7dbl_3_relu = nn.ReLU()
        self.features_12_branch7x7dbl_4_conv = nn.Conv2d(160, 160, kernel_size=[7, 1], stride=(1, 1), padding=(3, 0), bias=False)
        self.features_12_branch7x7dbl_4_bn = nn.BatchNorm2d(160, eps=0.001, momentum=0.1, affine=True)
        self.features_12_branch7x7dbl_4_relu = nn.ReLU()
        self.features_12_branch7x7dbl_5_conv = nn.Conv2d(160, 192, kernel_size=[1, 7], stride=(1, 1), padding=(0, 3), bias=False)
        self.features_12_branch7x7dbl_5_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_12_branch7x7dbl_5_relu = nn.ReLU()
        self.features_12_branch_avgpool = nn.AvgPool2d(kernel_size=[3, 3], stride=[1, 1], padding=1, ceil_mode=False, count_include_pad=False)
        self.features_12_branch_pool_conv = nn.Conv2d(768, 192, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_12_branch_pool_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_12_branch_pool_relu = nn.ReLU()
        self.features_13_branch1x1_conv = nn.Conv2d(768, 192, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_13_branch1x1_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_13_branch1x1_relu = nn.ReLU()
        self.features_13_branch7x7_1_conv = nn.Conv2d(768, 160, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_13_branch7x7_1_bn = nn.BatchNorm2d(160, eps=0.001, momentum=0.1, affine=True)
        self.features_13_branch7x7_1_relu = nn.ReLU()
        self.features_13_branch7x7_2_conv = nn.Conv2d(160, 160, kernel_size=[1, 7], stride=(1, 1), padding=(0, 3), bias=False)
        self.features_13_branch7x7_2_bn = nn.BatchNorm2d(160, eps=0.001, momentum=0.1, affine=True)
        self.features_13_branch7x7_2_relu = nn.ReLU()
        self.features_13_branch7x7_3_conv = nn.Conv2d(160, 192, kernel_size=[7, 1], stride=(1, 1), padding=(3, 0), bias=False)
        self.features_13_branch7x7_3_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_13_branch7x7_3_relu = nn.ReLU()
        self.features_13_branch7x7dbl_1_conv = nn.Conv2d(768, 160, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_13_branch7x7dbl_1_bn = nn.BatchNorm2d(160, eps=0.001, momentum=0.1, affine=True)
        self.features_13_branch7x7dbl_1_relu = nn.ReLU()
        self.features_13_branch7x7dbl_2_conv = nn.Conv2d(160, 160, kernel_size=[7, 1], stride=(1, 1), padding=(3, 0), bias=False)
        self.features_13_branch7x7dbl_2_bn = nn.BatchNorm2d(160, eps=0.001, momentum=0.1, affine=True)
        self.features_13_branch7x7dbl_2_relu = nn.ReLU()
        self.features_13_branch7x7dbl_3_conv = nn.Conv2d(160, 160, kernel_size=[1, 7], stride=(1, 1), padding=(0, 3), bias=False)
        self.features_13_branch7x7dbl_3_bn = nn.BatchNorm2d(160, eps=0.001, momentum=0.1, affine=True)
        self.features_13_branch7x7dbl_3_relu = nn.ReLU()
        self.features_13_branch7x7dbl_4_conv = nn.Conv2d(160, 160, kernel_size=[7, 1], stride=(1, 1), padding=(3, 0), bias=False)
        self.features_13_branch7x7dbl_4_bn = nn.BatchNorm2d(160, eps=0.001, momentum=0.1, affine=True)
        self.features_13_branch7x7dbl_4_relu = nn.ReLU()
        self.features_13_branch7x7dbl_5_conv = nn.Conv2d(160, 192, kernel_size=[1, 7], stride=(1, 1), padding=(0, 3), bias=False)
        self.features_13_branch7x7dbl_5_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_13_branch7x7dbl_5_relu = nn.ReLU()
        self.features_13_branch_avgpool = nn.AvgPool2d(kernel_size=[3, 3], stride=[1, 1], padding=1, ceil_mode=False, count_include_pad=False)
        self.features_13_branch_pool_conv = nn.Conv2d(768, 192, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_13_branch_pool_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_13_branch_pool_relu = nn.ReLU()
        self.features_14_branch1x1_conv = nn.Conv2d(768, 192, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_14_branch1x1_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_14_branch1x1_relu = nn.ReLU()
        self.features_14_branch7x7_1_conv = nn.Conv2d(768, 192, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_14_branch7x7_1_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_14_branch7x7_1_relu = nn.ReLU()
        self.features_14_branch7x7_2_conv = nn.Conv2d(192, 192, kernel_size=[1, 7], stride=(1, 1), padding=(0, 3), bias=False)
        self.features_14_branch7x7_2_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_14_branch7x7_2_relu = nn.ReLU()
        self.features_14_branch7x7_3_conv = nn.Conv2d(192, 192, kernel_size=[7, 1], stride=(1, 1), padding=(3, 0), bias=False)
        self.features_14_branch7x7_3_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_14_branch7x7_3_relu = nn.ReLU()
        self.features_14_branch7x7dbl_1_conv = nn.Conv2d(768, 192, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_14_branch7x7dbl_1_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_14_branch7x7dbl_1_relu = nn.ReLU()
        self.features_14_branch7x7dbl_2_conv = nn.Conv2d(192, 192, kernel_size=[7, 1], stride=(1, 1), padding=(3, 0), bias=False)
        self.features_14_branch7x7dbl_2_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_14_branch7x7dbl_2_relu = nn.ReLU()
        self.features_14_branch7x7dbl_3_conv = nn.Conv2d(192, 192, kernel_size=[1, 7], stride=(1, 1), padding=(0, 3), bias=False)
        self.features_14_branch7x7dbl_3_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_14_branch7x7dbl_3_relu = nn.ReLU()
        self.features_14_branch7x7dbl_4_conv = nn.Conv2d(192, 192, kernel_size=[7, 1], stride=(1, 1), padding=(3, 0), bias=False)
        self.features_14_branch7x7dbl_4_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_14_branch7x7dbl_4_relu = nn.ReLU()
        self.features_14_branch7x7dbl_5_conv = nn.Conv2d(192, 192, kernel_size=[1, 7], stride=(1, 1), padding=(0, 3), bias=False)
        self.features_14_branch7x7dbl_5_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_14_branch7x7dbl_5_relu = nn.ReLU()
        self.features_14_branch_avgpool = nn.AvgPool2d(kernel_size=[3, 3], stride=[1, 1], padding=1, ceil_mode=False, count_include_pad=False)
        self.features_14_branch_pool_conv = nn.Conv2d(768, 192, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_14_branch_pool_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_14_branch_pool_relu = nn.ReLU()
        self.features_15_branch3x3_1_conv = nn.Conv2d(768, 192, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_15_branch3x3_1_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_15_branch3x3_1_relu = nn.ReLU()
        self.features_15_branch3x3_2_conv = nn.Conv2d(192, 320, kernel_size=[3, 3], stride=(2, 2), bias=False)
        self.features_15_branch3x3_2_bn = nn.BatchNorm2d(320, eps=0.001, momentum=0.1, affine=True)
        self.features_15_branch3x3_2_relu = nn.ReLU()
        self.features_15_branch7x7x3_1_conv = nn.Conv2d(768, 192, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_15_branch7x7x3_1_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_15_branch7x7x3_1_relu = nn.ReLU()
        self.features_15_branch7x7x3_2_conv = nn.Conv2d(192, 192, kernel_size=[1, 7], stride=(1, 1), padding=(0, 3), bias=False)
        self.features_15_branch7x7x3_2_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_15_branch7x7x3_2_relu = nn.ReLU()
        self.features_15_branch7x7x3_3_conv = nn.Conv2d(192, 192, kernel_size=[7, 1], stride=(1, 1), padding=(3, 0), bias=False)
        self.features_15_branch7x7x3_3_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_15_branch7x7x3_3_relu = nn.ReLU()
        self.features_15_branch7x7x3_4_conv = nn.Conv2d(192, 192, kernel_size=[3, 3], stride=(2, 2), bias=False)
        self.features_15_branch7x7x3_4_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_15_branch7x7x3_4_relu = nn.ReLU()
        self.features_15_branch_maxpool = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
        self.features_16_branch1x1_conv = nn.Conv2d(1280, 320, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_16_branch1x1_bn = nn.BatchNorm2d(320, eps=0.001, momentum=0.1, affine=True)
        self.features_16_branch1x1_relu = nn.ReLU()
        self.features_16_branch3x3_1_conv = nn.Conv2d(1280, 384, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_16_branch3x3_1_bn = nn.BatchNorm2d(384, eps=0.001, momentum=0.1, affine=True)
        self.features_16_branch3x3_1_relu = nn.ReLU()
        self.features_16_branch3x3_2a_conv = nn.Conv2d(384, 384, kernel_size=[1, 3], stride=(1, 1), padding=(0, 1), bias=False)
        self.features_16_branch3x3_2a_bn = nn.BatchNorm2d(384, eps=0.001, momentum=0.1, affine=True)
        self.features_16_branch3x3_2a_relu = nn.ReLU()
        self.features_16_branch3x3_2b_conv = nn.Conv2d(384, 384, kernel_size=[3, 1], stride=(1, 1), padding=(1, 0), bias=False)
        self.features_16_branch3x3_2b_bn = nn.BatchNorm2d(384, eps=0.001, momentum=0.1, affine=True)
        self.features_16_branch3x3_2b_relu = nn.ReLU()
        self.features_16_branch3x3dbl_1_conv = nn.Conv2d(1280, 448, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_16_branch3x3dbl_1_bn = nn.BatchNorm2d(448, eps=0.001, momentum=0.1, affine=True)
        self.features_16_branch3x3dbl_1_relu = nn.ReLU()
        self.features_16_branch3x3dbl_2_conv = nn.Conv2d(448, 384, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1), bias=False)
        self.features_16_branch3x3dbl_2_bn = nn.BatchNorm2d(384, eps=0.001, momentum=0.1, affine=True)
        self.features_16_branch3x3dbl_2_relu = nn.ReLU()
        self.features_16_branch3x3dbl_3a_conv = nn.Conv2d(384, 384, kernel_size=[1, 3], stride=(1, 1), padding=(0, 1), bias=False)
        self.features_16_branch3x3dbl_3a_bn = nn.BatchNorm2d(384, eps=0.001, momentum=0.1, affine=True)
        self.features_16_branch3x3dbl_3a_relu = nn.ReLU()
        self.features_16_branch3x3dbl_3b_conv = nn.Conv2d(384, 384, kernel_size=[3, 1], stride=(1, 1), padding=(1, 0), bias=False)
        self.features_16_branch3x3dbl_3b_bn = nn.BatchNorm2d(384, eps=0.001, momentum=0.1, affine=True)
        self.features_16_branch3x3dbl_3b_relu = nn.ReLU()
        self.features_16_branch_avgpool = nn.AvgPool2d(kernel_size=[3, 3], stride=[1, 1], padding=1, ceil_mode=False, count_include_pad=False)
        self.features_16_branch_pool_conv = nn.Conv2d(1280, 192, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_16_branch_pool_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_16_branch_pool_relu = nn.ReLU()
        self.features_17_branch1x1_conv = nn.Conv2d(2048, 320, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_17_branch1x1_bn = nn.BatchNorm2d(320, eps=0.001, momentum=0.1, affine=True)
        self.features_17_branch1x1_relu = nn.ReLU()
        self.features_17_branch3x3_1_conv = nn.Conv2d(2048, 384, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_17_branch3x3_1_bn = nn.BatchNorm2d(384, eps=0.001, momentum=0.1, affine=True)
        self.features_17_branch3x3_1_relu = nn.ReLU()
        self.features_17_branch3x3_2a_conv = nn.Conv2d(384, 384, kernel_size=[1, 3], stride=(1, 1), padding=(0, 1), bias=False)
        self.features_17_branch3x3_2a_bn = nn.BatchNorm2d(384, eps=0.001, momentum=0.1, affine=True)
        self.features_17_branch3x3_2a_relu = nn.ReLU()
        self.features_17_branch3x3_2b_conv = nn.Conv2d(384, 384, kernel_size=[3, 1], stride=(1, 1), padding=(1, 0), bias=False)
        self.features_17_branch3x3_2b_bn = nn.BatchNorm2d(384, eps=0.001, momentum=0.1, affine=True)
        self.features_17_branch3x3_2b_relu = nn.ReLU()
        self.features_17_branch3x3dbl_1_conv = nn.Conv2d(2048, 448, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_17_branch3x3dbl_1_bn = nn.BatchNorm2d(448, eps=0.001, momentum=0.1, affine=True)
        self.features_17_branch3x3dbl_1_relu = nn.ReLU()
        self.features_17_branch3x3dbl_2_conv = nn.Conv2d(448, 384, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1), bias=False)
        self.features_17_branch3x3dbl_2_bn = nn.BatchNorm2d(384, eps=0.001, momentum=0.1, affine=True)
        self.features_17_branch3x3dbl_2_relu = nn.ReLU()
        self.features_17_branch3x3dbl_3a_conv = nn.Conv2d(384, 384, kernel_size=[1, 3], stride=(1, 1), padding=(0, 1), bias=False)
        self.features_17_branch3x3dbl_3a_bn = nn.BatchNorm2d(384, eps=0.001, momentum=0.1, affine=True)
        self.features_17_branch3x3dbl_3a_relu = nn.ReLU()
        self.features_17_branch3x3dbl_3b_conv = nn.Conv2d(384, 384, kernel_size=[3, 1], stride=(1, 1), padding=(1, 0), bias=False)
        self.features_17_branch3x3dbl_3b_bn = nn.BatchNorm2d(384, eps=0.001, momentum=0.1, affine=True)
        self.features_17_branch3x3dbl_3b_relu = nn.ReLU()
        self.features_17_branch_avgpool = nn.AvgPool2d(kernel_size=[3, 3], stride=[1, 1], padding=1, ceil_mode=False, count_include_pad=False)
        self.features_17_branch_pool_conv = nn.Conv2d(2048, 192, kernel_size=[1, 1], stride=(1, 1), bias=False)
        self.features_17_branch_pool_bn = nn.BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
        self.features_17_branch_pool_relu = nn.ReLU()
        self.features_18 = nn.AvgPool2d(kernel_size=[8, 8], stride=[8, 8], padding=0, ceil_mode=False, count_include_pad=False)
        self.features_19 = nn.Dropout(p=0.5)
        self.classifier_0 = nn.Linear(in_features=2048, out_features=1000, bias=True)

    def forward(self, data):
        features_0_conv = self.features_0_conv(data)
        features_0_bn = self.features_0_bn(features_0_conv)
        features_0_relu = self.features_0_relu(features_0_bn)
        features_1_conv = self.features_1_conv(features_0_relu)
        features_1_bn = self.features_1_bn(features_1_conv)
        features_1_relu = self.features_1_relu(features_1_bn)
        features_2_conv = self.features_2_conv(features_1_relu)
        features_2_bn = self.features_2_bn(features_2_conv)
        features_2_relu = self.features_2_relu(features_2_bn)
        features_3 = self.features_3(features_2_relu)
        features_4_conv = self.features_4_conv(features_3)
        features_4_bn = self.features_4_bn(features_4_conv)
        features_4_relu = self.features_4_relu(features_4_bn)
        features_5_conv = self.features_5_conv(features_4_relu)
        features_5_bn = self.features_5_bn(features_5_conv)
        features_5_relu = self.features_5_relu(features_5_bn)
        features_6 = self.features_6(features_5_relu)
        features_7_branch1x1_conv = self.features_7_branch1x1_conv(features_6)
        features_7_branch1x1_bn = self.features_7_branch1x1_bn(features_7_branch1x1_conv)
        features_7_branch1x1_relu = self.features_7_branch1x1_relu(features_7_branch1x1_bn)
        features_7_branch5x5_1_conv = self.features_7_branch5x5_1_conv(features_6)
        features_7_branch5x5_1_bn = self.features_7_branch5x5_1_bn(features_7_branch5x5_1_conv)
        features_7_branch5x5_1_relu = self.features_7_branch5x5_1_relu(features_7_branch5x5_1_bn)
        features_7_branch5x5_2_conv = self.features_7_branch5x5_2_conv(features_7_branch5x5_1_relu)
        features_7_branch5x5_2_bn = self.features_7_branch5x5_2_bn(features_7_branch5x5_2_conv)
        features_7_branch5x5_2_relu = self.features_7_branch5x5_2_relu(features_7_branch5x5_2_bn)
        features_7_branch3x3dbl_1_conv = self.features_7_branch3x3dbl_1_conv(features_6)
        features_7_branch3x3dbl_1_bn = self.features_7_branch3x3dbl_1_bn(features_7_branch3x3dbl_1_conv)
        features_7_branch3x3dbl_1_relu = self.features_7_branch3x3dbl_1_relu(features_7_branch3x3dbl_1_bn)
        features_7_branch3x3dbl_2_conv = self.features_7_branch3x3dbl_2_conv(features_7_branch3x3dbl_1_relu)
        features_7_branch3x3dbl_2_bn = self.features_7_branch3x3dbl_2_bn(features_7_branch3x3dbl_2_conv)
        features_7_branch3x3dbl_2_relu = self.features_7_branch3x3dbl_2_relu(features_7_branch3x3dbl_2_bn)
        features_7_branch3x3dbl_3_conv = self.features_7_branch3x3dbl_3_conv(features_7_branch3x3dbl_2_relu)
        features_7_branch3x3dbl_3_bn = self.features_7_branch3x3dbl_3_bn(features_7_branch3x3dbl_3_conv)
        features_7_branch3x3dbl_3_relu = self.features_7_branch3x3dbl_3_relu(features_7_branch3x3dbl_3_bn)
        features_7_branch_avgpool = self.features_7_branch_avgpool(features_6)
        features_7_branch_pool_conv = self.features_7_branch_pool_conv(features_7_branch_avgpool)
        features_7_branch_pool_bn = self.features_7_branch_pool_bn(features_7_branch_pool_conv)
        features_7_branch_pool_relu = self.features_7_branch_pool_relu(features_7_branch_pool_bn)
        features_7_cat = torch.cat((features_7_branch1x1_relu,features_7_branch5x5_2_relu,features_7_branch3x3dbl_3_relu,features_7_branch_pool_relu), dim=1)
        features_8_branch1x1_conv = self.features_8_branch1x1_conv(features_7_cat)
        features_8_branch1x1_bn = self.features_8_branch1x1_bn(features_8_branch1x1_conv)
        features_8_branch1x1_relu = self.features_8_branch1x1_relu(features_8_branch1x1_bn)
        features_8_branch5x5_1_conv = self.features_8_branch5x5_1_conv(features_7_cat)
        features_8_branch5x5_1_bn = self.features_8_branch5x5_1_bn(features_8_branch5x5_1_conv)
        features_8_branch5x5_1_relu = self.features_8_branch5x5_1_relu(features_8_branch5x5_1_bn)
        features_8_branch5x5_2_conv = self.features_8_branch5x5_2_conv(features_8_branch5x5_1_relu)
        features_8_branch5x5_2_bn = self.features_8_branch5x5_2_bn(features_8_branch5x5_2_conv)
        features_8_branch5x5_2_relu = self.features_8_branch5x5_2_relu(features_8_branch5x5_2_bn)
        features_8_branch3x3dbl_1_conv = self.features_8_branch3x3dbl_1_conv(features_7_cat)
        features_8_branch3x3dbl_1_bn = self.features_8_branch3x3dbl_1_bn(features_8_branch3x3dbl_1_conv)
        features_8_branch3x3dbl_1_relu = self.features_8_branch3x3dbl_1_relu(features_8_branch3x3dbl_1_bn)
        features_8_branch3x3dbl_2_conv = self.features_8_branch3x3dbl_2_conv(features_8_branch3x3dbl_1_relu)
        features_8_branch3x3dbl_2_bn = self.features_8_branch3x3dbl_2_bn(features_8_branch3x3dbl_2_conv)
        features_8_branch3x3dbl_2_relu = self.features_8_branch3x3dbl_2_relu(features_8_branch3x3dbl_2_bn)
        features_8_branch3x3dbl_3_conv = self.features_8_branch3x3dbl_3_conv(features_8_branch3x3dbl_2_relu)
        features_8_branch3x3dbl_3_bn = self.features_8_branch3x3dbl_3_bn(features_8_branch3x3dbl_3_conv)
        features_8_branch3x3dbl_3_relu = self.features_8_branch3x3dbl_3_relu(features_8_branch3x3dbl_3_bn)
        features_8_branch_avgpool = self.features_8_branch_avgpool(features_7_cat)
        features_8_branch_pool_conv = self.features_8_branch_pool_conv(features_8_branch_avgpool)
        features_8_branch_pool_bn = self.features_8_branch_pool_bn(features_8_branch_pool_conv)
        features_8_branch_pool_relu = self.features_8_branch_pool_relu(features_8_branch_pool_bn)
        features_8_cat = torch.cat((features_8_branch1x1_relu,features_8_branch5x5_2_relu,features_8_branch3x3dbl_3_relu,features_8_branch_pool_relu), dim=1)
        features_9_branch1x1_conv = self.features_9_branch1x1_conv(features_8_cat)
        features_9_branch1x1_bn = self.features_9_branch1x1_bn(features_9_branch1x1_conv)
        features_9_branch1x1_relu = self.features_9_branch1x1_relu(features_9_branch1x1_bn)
        features_9_branch5x5_1_conv = self.features_9_branch5x5_1_conv(features_8_cat)
        features_9_branch5x5_1_bn = self.features_9_branch5x5_1_bn(features_9_branch5x5_1_conv)
        features_9_branch5x5_1_relu = self.features_9_branch5x5_1_relu(features_9_branch5x5_1_bn)
        features_9_branch5x5_2_conv = self.features_9_branch5x5_2_conv(features_9_branch5x5_1_relu)
        features_9_branch5x5_2_bn = self.features_9_branch5x5_2_bn(features_9_branch5x5_2_conv)
        features_9_branch5x5_2_relu = self.features_9_branch5x5_2_relu(features_9_branch5x5_2_bn)
        features_9_branch3x3dbl_1_conv = self.features_9_branch3x3dbl_1_conv(features_8_cat)
        features_9_branch3x3dbl_1_bn = self.features_9_branch3x3dbl_1_bn(features_9_branch3x3dbl_1_conv)
        features_9_branch3x3dbl_1_relu = self.features_9_branch3x3dbl_1_relu(features_9_branch3x3dbl_1_bn)
        features_9_branch3x3dbl_2_conv = self.features_9_branch3x3dbl_2_conv(features_9_branch3x3dbl_1_relu)
        features_9_branch3x3dbl_2_bn = self.features_9_branch3x3dbl_2_bn(features_9_branch3x3dbl_2_conv)
        features_9_branch3x3dbl_2_relu = self.features_9_branch3x3dbl_2_relu(features_9_branch3x3dbl_2_bn)
        features_9_branch3x3dbl_3_conv = self.features_9_branch3x3dbl_3_conv(features_9_branch3x3dbl_2_relu)
        features_9_branch3x3dbl_3_bn = self.features_9_branch3x3dbl_3_bn(features_9_branch3x3dbl_3_conv)
        features_9_branch3x3dbl_3_relu = self.features_9_branch3x3dbl_3_relu(features_9_branch3x3dbl_3_bn)
        features_9_branch_avgpool = self.features_9_branch_avgpool(features_8_cat)
        features_9_branch_pool_conv = self.features_9_branch_pool_conv(features_9_branch_avgpool)
        features_9_branch_pool_bn = self.features_9_branch_pool_bn(features_9_branch_pool_conv)
        features_9_branch_pool_relu = self.features_9_branch_pool_relu(features_9_branch_pool_bn)
        features_9_cat = torch.cat((features_9_branch1x1_relu,features_9_branch5x5_2_relu,features_9_branch3x3dbl_3_relu,features_9_branch_pool_relu), dim=1)
        features_10_branch3x3_conv = self.features_10_branch3x3_conv(features_9_cat)
        features_10_branch3x3_bn = self.features_10_branch3x3_bn(features_10_branch3x3_conv)
        features_10_branch3x3_relu = self.features_10_branch3x3_relu(features_10_branch3x3_bn)
        features_10_branch3x3dbl_1_conv = self.features_10_branch3x3dbl_1_conv(features_9_cat)
        features_10_branch3x3dbl_1_bn = self.features_10_branch3x3dbl_1_bn(features_10_branch3x3dbl_1_conv)
        features_10_branch3x3dbl_1_relu = self.features_10_branch3x3dbl_1_relu(features_10_branch3x3dbl_1_bn)
        features_10_branch3x3dbl_2_conv = self.features_10_branch3x3dbl_2_conv(features_10_branch3x3dbl_1_relu)
        features_10_branch3x3dbl_2_bn = self.features_10_branch3x3dbl_2_bn(features_10_branch3x3dbl_2_conv)
        features_10_branch3x3dbl_2_relu = self.features_10_branch3x3dbl_2_relu(features_10_branch3x3dbl_2_bn)
        features_10_branch3x3dbl_3_conv = self.features_10_branch3x3dbl_3_conv(features_10_branch3x3dbl_2_relu)
        features_10_branch3x3dbl_3_bn = self.features_10_branch3x3dbl_3_bn(features_10_branch3x3dbl_3_conv)
        features_10_branch3x3dbl_3_relu = self.features_10_branch3x3dbl_3_relu(features_10_branch3x3dbl_3_bn)
        features_10_branch_pool = self.features_10_branch_pool(features_9_cat)
        features_10_cat = torch.cat((features_10_branch3x3_relu,features_10_branch3x3dbl_3_relu,features_10_branch_pool), dim=1)
        features_11_branch1x1_conv = self.features_11_branch1x1_conv(features_10_cat)
        features_11_branch1x1_bn = self.features_11_branch1x1_bn(features_11_branch1x1_conv)
        features_11_branch1x1_relu = self.features_11_branch1x1_relu(features_11_branch1x1_bn)
        features_11_branch7x7_1_conv = self.features_11_branch7x7_1_conv(features_10_cat)
        features_11_branch7x7_1_bn = self.features_11_branch7x7_1_bn(features_11_branch7x7_1_conv)
        features_11_branch7x7_1_relu = self.features_11_branch7x7_1_relu(features_11_branch7x7_1_bn)
        features_11_branch7x7_2_conv = self.features_11_branch7x7_2_conv(features_11_branch7x7_1_relu)
        features_11_branch7x7_2_bn = self.features_11_branch7x7_2_bn(features_11_branch7x7_2_conv)
        features_11_branch7x7_2_relu = self.features_11_branch7x7_2_relu(features_11_branch7x7_2_bn)
        features_11_branch7x7_3_conv = self.features_11_branch7x7_3_conv(features_11_branch7x7_2_relu)
        features_11_branch7x7_3_bn = self.features_11_branch7x7_3_bn(features_11_branch7x7_3_conv)
        features_11_branch7x7_3_relu = self.features_11_branch7x7_3_relu(features_11_branch7x7_3_bn)
        features_11_branch7x7dbl_1_conv = self.features_11_branch7x7dbl_1_conv(features_10_cat)
        features_11_branch7x7dbl_1_bn = self.features_11_branch7x7dbl_1_bn(features_11_branch7x7dbl_1_conv)
        features_11_branch7x7dbl_1_relu = self.features_11_branch7x7dbl_1_relu(features_11_branch7x7dbl_1_bn)
        features_11_branch7x7dbl_2_conv = self.features_11_branch7x7dbl_2_conv(features_11_branch7x7dbl_1_relu)
        features_11_branch7x7dbl_2_bn = self.features_11_branch7x7dbl_2_bn(features_11_branch7x7dbl_2_conv)
        features_11_branch7x7dbl_2_relu = self.features_11_branch7x7dbl_2_relu(features_11_branch7x7dbl_2_bn)
        features_11_branch7x7dbl_3_conv = self.features_11_branch7x7dbl_3_conv(features_11_branch7x7dbl_2_relu)
        features_11_branch7x7dbl_3_bn = self.features_11_branch7x7dbl_3_bn(features_11_branch7x7dbl_3_conv)
        features_11_branch7x7dbl_3_relu = self.features_11_branch7x7dbl_3_relu(features_11_branch7x7dbl_3_bn)
        features_11_branch7x7dbl_4_conv = self.features_11_branch7x7dbl_4_conv(features_11_branch7x7dbl_3_relu)
        features_11_branch7x7dbl_4_bn = self.features_11_branch7x7dbl_4_bn(features_11_branch7x7dbl_4_conv)
        features_11_branch7x7dbl_4_relu = self.features_11_branch7x7dbl_4_relu(features_11_branch7x7dbl_4_bn)
        features_11_branch7x7dbl_5_conv = self.features_11_branch7x7dbl_5_conv(features_11_branch7x7dbl_4_relu)
        features_11_branch7x7dbl_5_bn = self.features_11_branch7x7dbl_5_bn(features_11_branch7x7dbl_5_conv)
        features_11_branch7x7dbl_5_relu = self.features_11_branch7x7dbl_5_relu(features_11_branch7x7dbl_5_bn)
        features_11_branch_avgpool = self.features_11_branch_avgpool(features_10_cat)
        features_11_branch_pool_conv = self.features_11_branch_pool_conv(features_11_branch_avgpool)
        features_11_branch_pool_bn = self.features_11_branch_pool_bn(features_11_branch_pool_conv)
        features_11_branch_pool_relu = self.features_11_branch_pool_relu(features_11_branch_pool_bn)
        features_11_cat = torch.cat((features_11_branch1x1_relu,features_11_branch7x7_3_relu,features_11_branch7x7dbl_5_relu,features_11_branch_pool_relu), dim=1)
        features_12_branch1x1_conv = self.features_12_branch1x1_conv(features_11_cat)
        features_12_branch1x1_bn = self.features_12_branch1x1_bn(features_12_branch1x1_conv)
        features_12_branch1x1_relu = self.features_12_branch1x1_relu(features_12_branch1x1_bn)
        features_12_branch7x7_1_conv = self.features_12_branch7x7_1_conv(features_11_cat)
        features_12_branch7x7_1_bn = self.features_12_branch7x7_1_bn(features_12_branch7x7_1_conv)
        features_12_branch7x7_1_relu = self.features_12_branch7x7_1_relu(features_12_branch7x7_1_bn)
        features_12_branch7x7_2_conv = self.features_12_branch7x7_2_conv(features_12_branch7x7_1_relu)
        features_12_branch7x7_2_bn = self.features_12_branch7x7_2_bn(features_12_branch7x7_2_conv)
        features_12_branch7x7_2_relu = self.features_12_branch7x7_2_relu(features_12_branch7x7_2_bn)
        features_12_branch7x7_3_conv = self.features_12_branch7x7_3_conv(features_12_branch7x7_2_relu)
        features_12_branch7x7_3_bn = self.features_12_branch7x7_3_bn(features_12_branch7x7_3_conv)
        features_12_branch7x7_3_relu = self.features_12_branch7x7_3_relu(features_12_branch7x7_3_bn)
        features_12_branch7x7dbl_1_conv = self.features_12_branch7x7dbl_1_conv(features_11_cat)
        features_12_branch7x7dbl_1_bn = self.features_12_branch7x7dbl_1_bn(features_12_branch7x7dbl_1_conv)
        features_12_branch7x7dbl_1_relu = self.features_12_branch7x7dbl_1_relu(features_12_branch7x7dbl_1_bn)
        features_12_branch7x7dbl_2_conv = self.features_12_branch7x7dbl_2_conv(features_12_branch7x7dbl_1_relu)
        features_12_branch7x7dbl_2_bn = self.features_12_branch7x7dbl_2_bn(features_12_branch7x7dbl_2_conv)
        features_12_branch7x7dbl_2_relu = self.features_12_branch7x7dbl_2_relu(features_12_branch7x7dbl_2_bn)
        features_12_branch7x7dbl_3_conv = self.features_12_branch7x7dbl_3_conv(features_12_branch7x7dbl_2_relu)
        features_12_branch7x7dbl_3_bn = self.features_12_branch7x7dbl_3_bn(features_12_branch7x7dbl_3_conv)
        features_12_branch7x7dbl_3_relu = self.features_12_branch7x7dbl_3_relu(features_12_branch7x7dbl_3_bn)
        features_12_branch7x7dbl_4_conv = self.features_12_branch7x7dbl_4_conv(features_12_branch7x7dbl_3_relu)
        features_12_branch7x7dbl_4_bn = self.features_12_branch7x7dbl_4_bn(features_12_branch7x7dbl_4_conv)
        features_12_branch7x7dbl_4_relu = self.features_12_branch7x7dbl_4_relu(features_12_branch7x7dbl_4_bn)
        features_12_branch7x7dbl_5_conv = self.features_12_branch7x7dbl_5_conv(features_12_branch7x7dbl_4_relu)
        features_12_branch7x7dbl_5_bn = self.features_12_branch7x7dbl_5_bn(features_12_branch7x7dbl_5_conv)
        features_12_branch7x7dbl_5_relu = self.features_12_branch7x7dbl_5_relu(features_12_branch7x7dbl_5_bn)
        features_12_branch_avgpool = self.features_12_branch_avgpool(features_11_cat)
        features_12_branch_pool_conv = self.features_12_branch_pool_conv(features_12_branch_avgpool)
        features_12_branch_pool_bn = self.features_12_branch_pool_bn(features_12_branch_pool_conv)
        features_12_branch_pool_relu = self.features_12_branch_pool_relu(features_12_branch_pool_bn)
        features_12_cat = torch.cat((features_12_branch1x1_relu,features_12_branch7x7_3_relu,features_12_branch7x7dbl_5_relu,features_12_branch_pool_relu), dim=1)
        features_13_branch1x1_conv = self.features_13_branch1x1_conv(features_12_cat)
        features_13_branch1x1_bn = self.features_13_branch1x1_bn(features_13_branch1x1_conv)
        features_13_branch1x1_relu = self.features_13_branch1x1_relu(features_13_branch1x1_bn)
        features_13_branch7x7_1_conv = self.features_13_branch7x7_1_conv(features_12_cat)
        features_13_branch7x7_1_bn = self.features_13_branch7x7_1_bn(features_13_branch7x7_1_conv)
        features_13_branch7x7_1_relu = self.features_13_branch7x7_1_relu(features_13_branch7x7_1_bn)
        features_13_branch7x7_2_conv = self.features_13_branch7x7_2_conv(features_13_branch7x7_1_relu)
        features_13_branch7x7_2_bn = self.features_13_branch7x7_2_bn(features_13_branch7x7_2_conv)
        features_13_branch7x7_2_relu = self.features_13_branch7x7_2_relu(features_13_branch7x7_2_bn)
        features_13_branch7x7_3_conv = self.features_13_branch7x7_3_conv(features_13_branch7x7_2_relu)
        features_13_branch7x7_3_bn = self.features_13_branch7x7_3_bn(features_13_branch7x7_3_conv)
        features_13_branch7x7_3_relu = self.features_13_branch7x7_3_relu(features_13_branch7x7_3_bn)
        features_13_branch7x7dbl_1_conv = self.features_13_branch7x7dbl_1_conv(features_12_cat)
        features_13_branch7x7dbl_1_bn = self.features_13_branch7x7dbl_1_bn(features_13_branch7x7dbl_1_conv)
        features_13_branch7x7dbl_1_relu = self.features_13_branch7x7dbl_1_relu(features_13_branch7x7dbl_1_bn)
        features_13_branch7x7dbl_2_conv = self.features_13_branch7x7dbl_2_conv(features_13_branch7x7dbl_1_relu)
        features_13_branch7x7dbl_2_bn = self.features_13_branch7x7dbl_2_bn(features_13_branch7x7dbl_2_conv)
        features_13_branch7x7dbl_2_relu = self.features_13_branch7x7dbl_2_relu(features_13_branch7x7dbl_2_bn)
        features_13_branch7x7dbl_3_conv = self.features_13_branch7x7dbl_3_conv(features_13_branch7x7dbl_2_relu)
        features_13_branch7x7dbl_3_bn = self.features_13_branch7x7dbl_3_bn(features_13_branch7x7dbl_3_conv)
        features_13_branch7x7dbl_3_relu = self.features_13_branch7x7dbl_3_relu(features_13_branch7x7dbl_3_bn)
        features_13_branch7x7dbl_4_conv = self.features_13_branch7x7dbl_4_conv(features_13_branch7x7dbl_3_relu)
        features_13_branch7x7dbl_4_bn = self.features_13_branch7x7dbl_4_bn(features_13_branch7x7dbl_4_conv)
        features_13_branch7x7dbl_4_relu = self.features_13_branch7x7dbl_4_relu(features_13_branch7x7dbl_4_bn)
        features_13_branch7x7dbl_5_conv = self.features_13_branch7x7dbl_5_conv(features_13_branch7x7dbl_4_relu)
        features_13_branch7x7dbl_5_bn = self.features_13_branch7x7dbl_5_bn(features_13_branch7x7dbl_5_conv)
        features_13_branch7x7dbl_5_relu = self.features_13_branch7x7dbl_5_relu(features_13_branch7x7dbl_5_bn)
        features_13_branch_avgpool = self.features_13_branch_avgpool(features_12_cat)
        features_13_branch_pool_conv = self.features_13_branch_pool_conv(features_13_branch_avgpool)
        features_13_branch_pool_bn = self.features_13_branch_pool_bn(features_13_branch_pool_conv)
        features_13_branch_pool_relu = self.features_13_branch_pool_relu(features_13_branch_pool_bn)
        features_13_cat = torch.cat((features_13_branch1x1_relu,features_13_branch7x7_3_relu,features_13_branch7x7dbl_5_relu,features_13_branch_pool_relu), dim=1)
        features_14_branch1x1_conv = self.features_14_branch1x1_conv(features_13_cat)
        features_14_branch1x1_bn = self.features_14_branch1x1_bn(features_14_branch1x1_conv)
        features_14_branch1x1_relu = self.features_14_branch1x1_relu(features_14_branch1x1_bn)
        features_14_branch7x7_1_conv = self.features_14_branch7x7_1_conv(features_13_cat)
        features_14_branch7x7_1_bn = self.features_14_branch7x7_1_bn(features_14_branch7x7_1_conv)
        features_14_branch7x7_1_relu = self.features_14_branch7x7_1_relu(features_14_branch7x7_1_bn)
        features_14_branch7x7_2_conv = self.features_14_branch7x7_2_conv(features_14_branch7x7_1_relu)
        features_14_branch7x7_2_bn = self.features_14_branch7x7_2_bn(features_14_branch7x7_2_conv)
        features_14_branch7x7_2_relu = self.features_14_branch7x7_2_relu(features_14_branch7x7_2_bn)
        features_14_branch7x7_3_conv = self.features_14_branch7x7_3_conv(features_14_branch7x7_2_relu)
        features_14_branch7x7_3_bn = self.features_14_branch7x7_3_bn(features_14_branch7x7_3_conv)
        features_14_branch7x7_3_relu = self.features_14_branch7x7_3_relu(features_14_branch7x7_3_bn)
        features_14_branch7x7dbl_1_conv = self.features_14_branch7x7dbl_1_conv(features_13_cat)
        features_14_branch7x7dbl_1_bn = self.features_14_branch7x7dbl_1_bn(features_14_branch7x7dbl_1_conv)
        features_14_branch7x7dbl_1_relu = self.features_14_branch7x7dbl_1_relu(features_14_branch7x7dbl_1_bn)
        features_14_branch7x7dbl_2_conv = self.features_14_branch7x7dbl_2_conv(features_14_branch7x7dbl_1_relu)
        features_14_branch7x7dbl_2_bn = self.features_14_branch7x7dbl_2_bn(features_14_branch7x7dbl_2_conv)
        features_14_branch7x7dbl_2_relu = self.features_14_branch7x7dbl_2_relu(features_14_branch7x7dbl_2_bn)
        features_14_branch7x7dbl_3_conv = self.features_14_branch7x7dbl_3_conv(features_14_branch7x7dbl_2_relu)
        features_14_branch7x7dbl_3_bn = self.features_14_branch7x7dbl_3_bn(features_14_branch7x7dbl_3_conv)
        features_14_branch7x7dbl_3_relu = self.features_14_branch7x7dbl_3_relu(features_14_branch7x7dbl_3_bn)
        features_14_branch7x7dbl_4_conv = self.features_14_branch7x7dbl_4_conv(features_14_branch7x7dbl_3_relu)
        features_14_branch7x7dbl_4_bn = self.features_14_branch7x7dbl_4_bn(features_14_branch7x7dbl_4_conv)
        features_14_branch7x7dbl_4_relu = self.features_14_branch7x7dbl_4_relu(features_14_branch7x7dbl_4_bn)
        features_14_branch7x7dbl_5_conv = self.features_14_branch7x7dbl_5_conv(features_14_branch7x7dbl_4_relu)
        features_14_branch7x7dbl_5_bn = self.features_14_branch7x7dbl_5_bn(features_14_branch7x7dbl_5_conv)
        features_14_branch7x7dbl_5_relu = self.features_14_branch7x7dbl_5_relu(features_14_branch7x7dbl_5_bn)
        features_14_branch_avgpool = self.features_14_branch_avgpool(features_13_cat)
        features_14_branch_pool_conv = self.features_14_branch_pool_conv(features_14_branch_avgpool)
        features_14_branch_pool_bn = self.features_14_branch_pool_bn(features_14_branch_pool_conv)
        features_14_branch_pool_relu = self.features_14_branch_pool_relu(features_14_branch_pool_bn)
        features_14_cat = torch.cat((features_14_branch1x1_relu,features_14_branch7x7_3_relu,features_14_branch7x7dbl_5_relu,features_14_branch_pool_relu), dim=1)
        features_15_branch3x3_1_conv = self.features_15_branch3x3_1_conv(features_14_cat)
        features_15_branch3x3_1_bn = self.features_15_branch3x3_1_bn(features_15_branch3x3_1_conv)
        features_15_branch3x3_1_relu = self.features_15_branch3x3_1_relu(features_15_branch3x3_1_bn)
        features_15_branch3x3_2_conv = self.features_15_branch3x3_2_conv(features_15_branch3x3_1_relu)
        features_15_branch3x3_2_bn = self.features_15_branch3x3_2_bn(features_15_branch3x3_2_conv)
        features_15_branch3x3_2_relu = self.features_15_branch3x3_2_relu(features_15_branch3x3_2_bn)
        features_15_branch7x7x3_1_conv = self.features_15_branch7x7x3_1_conv(features_14_cat)
        features_15_branch7x7x3_1_bn = self.features_15_branch7x7x3_1_bn(features_15_branch7x7x3_1_conv)
        features_15_branch7x7x3_1_relu = self.features_15_branch7x7x3_1_relu(features_15_branch7x7x3_1_bn)
        features_15_branch7x7x3_2_conv = self.features_15_branch7x7x3_2_conv(features_15_branch7x7x3_1_relu)
        features_15_branch7x7x3_2_bn = self.features_15_branch7x7x3_2_bn(features_15_branch7x7x3_2_conv)
        features_15_branch7x7x3_2_relu = self.features_15_branch7x7x3_2_relu(features_15_branch7x7x3_2_bn)
        features_15_branch7x7x3_3_conv = self.features_15_branch7x7x3_3_conv(features_15_branch7x7x3_2_relu)
        features_15_branch7x7x3_3_bn = self.features_15_branch7x7x3_3_bn(features_15_branch7x7x3_3_conv)
        features_15_branch7x7x3_3_relu = self.features_15_branch7x7x3_3_relu(features_15_branch7x7x3_3_bn)
        features_15_branch7x7x3_4_conv = self.features_15_branch7x7x3_4_conv(features_15_branch7x7x3_3_relu)
        features_15_branch7x7x3_4_bn = self.features_15_branch7x7x3_4_bn(features_15_branch7x7x3_4_conv)
        features_15_branch7x7x3_4_relu = self.features_15_branch7x7x3_4_relu(features_15_branch7x7x3_4_bn)
        features_15_branch_maxpool = self.features_15_branch_maxpool(features_14_cat)
        features_15_cat = torch.cat((features_15_branch3x3_2_relu,features_15_branch7x7x3_4_relu,features_15_branch_maxpool), dim=1)
        features_16_branch1x1_conv = self.features_16_branch1x1_conv(features_15_cat)
        features_16_branch1x1_bn = self.features_16_branch1x1_bn(features_16_branch1x1_conv)
        features_16_branch1x1_relu = self.features_16_branch1x1_relu(features_16_branch1x1_bn)
        features_16_branch3x3_1_conv = self.features_16_branch3x3_1_conv(features_15_cat)
        features_16_branch3x3_1_bn = self.features_16_branch3x3_1_bn(features_16_branch3x3_1_conv)
        features_16_branch3x3_1_relu = self.features_16_branch3x3_1_relu(features_16_branch3x3_1_bn)
        features_16_branch3x3_2a_conv = self.features_16_branch3x3_2a_conv(features_16_branch3x3_1_relu)
        features_16_branch3x3_2a_bn = self.features_16_branch3x3_2a_bn(features_16_branch3x3_2a_conv)
        features_16_branch3x3_2a_relu = self.features_16_branch3x3_2a_relu(features_16_branch3x3_2a_bn)
        features_16_branch3x3_2b_conv = self.features_16_branch3x3_2b_conv(features_16_branch3x3_1_relu)
        features_16_branch3x3_2b_bn = self.features_16_branch3x3_2b_bn(features_16_branch3x3_2b_conv)
        features_16_branch3x3_2b_relu = self.features_16_branch3x3_2b_relu(features_16_branch3x3_2b_bn)
        features_16_branch2_cat = torch.cat((features_16_branch3x3_2a_relu,features_16_branch3x3_2b_relu), dim=1)
        features_16_branch3x3dbl_1_conv = self.features_16_branch3x3dbl_1_conv(features_15_cat)
        features_16_branch3x3dbl_1_bn = self.features_16_branch3x3dbl_1_bn(features_16_branch3x3dbl_1_conv)
        features_16_branch3x3dbl_1_relu = self.features_16_branch3x3dbl_1_relu(features_16_branch3x3dbl_1_bn)
        features_16_branch3x3dbl_2_conv = self.features_16_branch3x3dbl_2_conv(features_16_branch3x3dbl_1_relu)
        features_16_branch3x3dbl_2_bn = self.features_16_branch3x3dbl_2_bn(features_16_branch3x3dbl_2_conv)
        features_16_branch3x3dbl_2_relu = self.features_16_branch3x3dbl_2_relu(features_16_branch3x3dbl_2_bn)
        features_16_branch3x3dbl_3a_conv = self.features_16_branch3x3dbl_3a_conv(features_16_branch3x3dbl_2_relu)
        features_16_branch3x3dbl_3a_bn = self.features_16_branch3x3dbl_3a_bn(features_16_branch3x3dbl_3a_conv)
        features_16_branch3x3dbl_3a_relu = self.features_16_branch3x3dbl_3a_relu(features_16_branch3x3dbl_3a_bn)
        features_16_branch3x3dbl_3b_conv = self.features_16_branch3x3dbl_3b_conv(features_16_branch3x3dbl_2_relu)
        features_16_branch3x3dbl_3b_bn = self.features_16_branch3x3dbl_3b_bn(features_16_branch3x3dbl_3b_conv)
        features_16_branch3x3dbl_3b_relu = self.features_16_branch3x3dbl_3b_relu(features_16_branch3x3dbl_3b_bn)
        features_16_branch3_cat = torch.cat((features_16_branch3x3dbl_3a_relu,features_16_branch3x3dbl_3b_relu), dim=1)
        features_16_branch_avgpool = self.features_16_branch_avgpool(features_15_cat)
        features_16_branch_pool_conv = self.features_16_branch_pool_conv(features_16_branch_avgpool)
        features_16_branch_pool_bn = self.features_16_branch_pool_bn(features_16_branch_pool_conv)
        features_16_branch_pool_relu = self.features_16_branch_pool_relu(features_16_branch_pool_bn)
        features_16_cat = torch.cat((features_16_branch1x1_relu,features_16_branch2_cat,features_16_branch3_cat,features_16_branch_pool_relu), dim=1)
        features_17_branch1x1_conv = self.features_17_branch1x1_conv(features_16_cat)
        features_17_branch1x1_bn = self.features_17_branch1x1_bn(features_17_branch1x1_conv)
        features_17_branch1x1_relu = self.features_17_branch1x1_relu(features_17_branch1x1_bn)
        features_17_branch3x3_1_conv = self.features_17_branch3x3_1_conv(features_16_cat)
        features_17_branch3x3_1_bn = self.features_17_branch3x3_1_bn(features_17_branch3x3_1_conv)
        features_17_branch3x3_1_relu = self.features_17_branch3x3_1_relu(features_17_branch3x3_1_bn)
        features_17_branch3x3_2a_conv = self.features_17_branch3x3_2a_conv(features_17_branch3x3_1_relu)
        features_17_branch3x3_2a_bn = self.features_17_branch3x3_2a_bn(features_17_branch3x3_2a_conv)
        features_17_branch3x3_2a_relu = self.features_17_branch3x3_2a_relu(features_17_branch3x3_2a_bn)
        features_17_branch3x3_2b_conv = self.features_17_branch3x3_2b_conv(features_17_branch3x3_1_relu)
        features_17_branch3x3_2b_bn = self.features_17_branch3x3_2b_bn(features_17_branch3x3_2b_conv)
        features_17_branch3x3_2b_relu = self.features_17_branch3x3_2b_relu(features_17_branch3x3_2b_bn)
        features_17_branch2_cat = torch.cat((features_17_branch3x3_2a_relu,features_17_branch3x3_2b_relu), dim=1)
        features_17_branch3x3dbl_1_conv = self.features_17_branch3x3dbl_1_conv(features_16_cat)
        features_17_branch3x3dbl_1_bn = self.features_17_branch3x3dbl_1_bn(features_17_branch3x3dbl_1_conv)
        features_17_branch3x3dbl_1_relu = self.features_17_branch3x3dbl_1_relu(features_17_branch3x3dbl_1_bn)
        features_17_branch3x3dbl_2_conv = self.features_17_branch3x3dbl_2_conv(features_17_branch3x3dbl_1_relu)
        features_17_branch3x3dbl_2_bn = self.features_17_branch3x3dbl_2_bn(features_17_branch3x3dbl_2_conv)
        features_17_branch3x3dbl_2_relu = self.features_17_branch3x3dbl_2_relu(features_17_branch3x3dbl_2_bn)
        features_17_branch3x3dbl_3a_conv = self.features_17_branch3x3dbl_3a_conv(features_17_branch3x3dbl_2_relu)
        features_17_branch3x3dbl_3a_bn = self.features_17_branch3x3dbl_3a_bn(features_17_branch3x3dbl_3a_conv)
        features_17_branch3x3dbl_3a_relu = self.features_17_branch3x3dbl_3a_relu(features_17_branch3x3dbl_3a_bn)
        features_17_branch3x3dbl_3b_conv = self.features_17_branch3x3dbl_3b_conv(features_17_branch3x3dbl_2_relu)
        features_17_branch3x3dbl_3b_bn = self.features_17_branch3x3dbl_3b_bn(features_17_branch3x3dbl_3b_conv)
        features_17_branch3x3dbl_3b_relu = self.features_17_branch3x3dbl_3b_relu(features_17_branch3x3dbl_3b_bn)
        features_17_branch3_cat = torch.cat((features_17_branch3x3dbl_3a_relu,features_17_branch3x3dbl_3b_relu), dim=1)
        features_17_branch_avgpool = self.features_17_branch_avgpool(features_16_cat)
        features_17_branch_pool_conv = self.features_17_branch_pool_conv(features_17_branch_avgpool)
        features_17_branch_pool_bn = self.features_17_branch_pool_bn(features_17_branch_pool_conv)
        features_17_branch_pool_relu = self.features_17_branch_pool_relu(features_17_branch_pool_bn)
        features_17_cat = torch.cat((features_17_branch1x1_relu,features_17_branch2_cat,features_17_branch3_cat,features_17_branch_pool_relu), dim=1)
        features_18 = self.features_18(features_17_cat)
#         features_19 = self.features_19(features_18)
        classifier_flatten = features_18.view(features_18.size(0), -1)
#         classifier_0 = self.classifier_0(classifier_flatten)
        return classifier_flatten

def inception_v3_pt_mcn(weights_path=None, **kwargs):
    """
    load imported model instance

    Args:
        weights_path (str): If set, loads model weights from the given path
    """
    model = Inception_v3_pt_mcn()
    if weights_path:
        state_dict = torch.load(weights_path)
        model.load_state_dict(state_dict)
    return model

In [ ]:
cnn_encoder = inception_v3_pt_mcn(weights_path='../input/pytorch-inception-v3/inception_v3_pt_mcn.pth')

In [ ]:
clip2 = train_dataset.__getitem__(0)[0]
cnn_encoder(clip2[0].unsqueeze(0)).shape

# RNN Decoder

This RNN will receive an array of encoded spatial data. So for each of the 200 frames, there will be a tensor of shape 1x2048 **Input shape** 200 x 1 x 2048 **Output** 1x40

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, CNN_embed_dim=2048, h_RNN_layers=200, h_RNN=512, h_FC_dim=256, drop_p=0.3, num_classes=40):
        super(DecoderRNN, self).__init__()

        self.RNN_input_size = CNN_embed_dim
        self.h_RNN_layers = h_RNN_layers   # RNN hidden layers
        self.h_RNN = h_RNN                 # RNN hidden nodes
        self.h_FC_dim = h_FC_dim
        self.drop_p = drop_p
        self.num_classes = num_classes

        self.LSTM = nn.LSTM(
            input_size=self.RNN_input_size,
            hidden_size=self.h_RNN,        
            num_layers=h_RNN_layers,       
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )

        self.fc1 = nn.Linear(self.h_RNN, self.h_FC_dim)
        self.fc2 = nn.Linear(self.h_FC_dim, self.num_classes)

    def forward(self, x_RNN):
        
        self.LSTM.flatten_parameters()
        RNN_out, (h_n, h_c) = self.LSTM(x_RNN, None)  
        """ h_n shape (n_layers, batch, hidden_size), h_c shape (n_layers, batch, hidden_size) """ 
        """ None represents zero initial hidden state. RNN_out has shape=(batch, time_step, output_size) """

        # FC layers
        x = self.fc1(RNN_out[:, -1, :])   # choose RNN_out at the last time step
        x = F.relu(x)
        x = F.dropout(x, p=self.drop_p, training=self.training)
        x = self.fc2(x)
        last_output = x[-1].unsqueeze(0)
        return last_output

In [ ]:
rnn_decoder = DecoderRNN(drop_p=0.0)

In [ ]:
if train_on_gpu:
    cnn_encoder = cnn_encoder.cuda()
    rnn_decoder = rnn_decoder.cuda()

# Optimizer

In [ ]:
optimizer = torch.optim.Adam(rnn_decoder.parameters(), lr=0.001)

# Model Training

In [ ]:
train_counter = []
train_loss_history = []
train_iteration_number= 0

valid_counter = []
valid_loss_history = []
valid_iteration_number= 0

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

train_class_correct = list(0 for i in range(40))
train_class_total = list(0 for i in range(40))

valid_class_correct = list(0 for i in range(40))
valid_class_total = list(0 for i in range(40))

In [ ]:
# spatial_data2 = torch.zeros(200,1,1000)
# spatial_data2 = spatial_data2.cuda()
# o = rnn_decoder(spatial_data2)
# print(o.shape)
# # y_pred = torch.max(o, 1)
# # y_pred

In [ ]:
N_count = 0   # counting total trained sample in one epoch
log_interval = 1

for epoch in range(0,Config.train_number_epochs):
    rnn_decoder.train()
    for batch, (clip, label) in enumerate(train_loader):
        # distribute data to device
        spatial_data = torch.zeros(200,1,2048)
        spatial_data = spatial_data.cuda()
        clip = clip.cuda()
        label = label.cuda()
        N_count += clip.size(0)
        optimizer.zero_grad()

        count=0
        for i in clip[0]:
            spatial_frame = i
            spatial_data[count,:,:] = cnn_encoder(spatial_frame.unsqueeze(0))
            count+=1

        output = rnn_decoder(spatial_data)   # output has dim = (batch, number of classes)

        loss = F.cross_entropy(output, label)

        pred = torch.max(output, 1)[1]

        loss.backward()
        optimizer.step()

        correct = pred.eq(label.view_as(pred))
        for j in range(len(label)):
            target = label[j].data
            train_class_correct[target] += correct[j].item()
            train_class_total[target] += 1

        if batch%1 == 0:
            print("Clip number {}\n Current loss {}\n".format(batch+1,loss.item()))
            train_iteration_number +=1
            train_counter.append(train_iteration_number)
            train_loss_history.append(loss.item())

#             for i in range(40):
#                 if train_class_total[i] > 0:
#                     print('\nTraining Accuracy of %5s: %2d%% (%2d/%2d)' % (
#                         str(i), 100 * train_class_correct[i] / train_class_total[i],
#                         np.sum(train_class_correct[i]), np.sum(train_class_total[i])))

            print('\nTraining Accuracy (Overall): %2d%% (%2d/%2d)' % (
                100. * np.sum(train_class_correct) / np.sum(train_class_total),
                np.sum(train_class_correct), np.sum(train_class_total)))
            
    rnn_decoder.eval()
    for batch, (clip, label) in enumerate(valid_loader):
        # distribute data to device
        spatial_data = torch.zeros(200,1,2048)
        spatial_data = spatial_data.cuda()
        clip = clip.cuda()
        label = label.cuda()
        N_count += clip.size(0)

        count=0
        for i in clip[0]:
            spatial_frame = i
            spatial_data[count,:,:] = cnn_encoder(spatial_frame.unsqueeze(0))
            count+=1

        output = rnn_decoder(spatial_data)   # output has dim = (batch, number of classes)

        loss = F.cross_entropy(output, label)

        pred = torch.max(output, 1)[1]

        correct = pred.eq(label.view_as(pred))
        for j in range(len(label)):
            target = label[j].data
            valid_class_correct[target] += correct[j].item()
            valid_class_total[target] += 1

        if batch%1 == 0:
            print("Clip number {}\n Current loss {}\n".format(batch+1,loss.item()))
            valid_iteration_number +=1
            valid_counter.append(train_iteration_number)
            valid_loss_history.append(loss.item())

#             for i in range(40):
#                 if valid_class_total[i] > 0:
#                     print('\nValidation Accuracy of %5s: %2d%% (%2d/%2d)' % (
#                         str(i), 100 * valid_class_correct[i] / valid_class_total[i],
#                         np.sum(valid_class_correct[i]), np.sum(valid_class_total[i])))

            print('\nValidation Accuracy (Overall): %2d%% (%2d/%2d)' % (
                100. * np.sum(valid_class_correct) / np.sum(valid_class_total),
                np.sum(valid_class_correct), np.sum(valid_class_total)))

# Testing Model